# Achieve video understanding using Generative AI on AWS

This workshop focuses on contextualizing long-form video assets, which are among the most challenging media types to analyze. We will systematically demonstrate how to break down lengthy videos into logical segments and then generate relevant insights for each segment using generative AI on AWS. This process enables a granular understanding of long-form video content at the scene level. By completing this workshop, you will learn our video understanding techniques that allow users not only to comprehend and quickly find specific videos but also to understand and identify relevant scenes or clips within those videos.

## Workflow steps

1. Setup prequisite and upload a media asset to Amazon Simple Storage Service (S3).
2. Generate the audio chapter points: we use Amazon Transcribe, Automatic Speech Recognition (ASR) service to generate transcription from the audio dialogues of the media asset. then use Anthropic's Claude 3 Haiku model to analyze the conversation and identify chapter points based on significantly topic changes.
3. In parallel, generate scene grid from video frames: we sample the frames from video and use Amazon Titan Multimodal Embedding model to help group frames into shots and then group shots into scenes based on visual similarity.
4. Align scene and audio chapter: align video scenes with the audio chapters to identify un-intrusive breaks for ad insertion
5. Generate the contextual response: We send the the scene grid, transcription, to Anthropic Claude 3 model in Amazon Bedrock to generate relevant contextual response: such as scene description, sentiment, relevant IAB or any other custom taxonomy.

![workflow](static/workflow.png)

## Pre-req
You must run the [workshop_setup.ipynb](../lab00-setup/workshop_setup.ipynb) notebook in `lab00-setup` before starting this lab.

In [ ]:
import warnings
warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook")

## Load the parameters

In [ ]:
print("load bucket, region, and role....\n")
# bucket and parameter stored from Initial setup lab00
%store -r bucket
%store -r role
%store -r region

## check all 5 values are printed and do not fail
print(bucket)
print(role)
print(region)

In [ ]:
from IPython.display import JSON
from IPython.display import Video
from IPython.display import Pretty
import json
import boto3
import sagemaker
from termcolor import colored
import time
import datetime
from pathlib import Path
from termcolor import colored
import glob
import os
from functools import cmp_to_key
from lib import transcribe_helper as trh
from lib import s3_helper as s3h
from lib import chapters as chpt
from lib import util
from lib import embeddings
from lib import frames
from lib import ffmpeg_helper as ffh
from lib import bedrock_helper as brh

### Download the sample video, Meridian, from Netflix

The open source content is available under the [Creative Commons Attribution 4.0 International Public License](https://creativecommons.org/licenses/by/4.0/legalcode)

In [ ]:
from pathlib import Path

mp4_file = 'Netflix_Open_Content_Meridian.mp4'
video_dir = Path(mp4_file).stem

url = f"https://dx2y1cac29mt3.cloudfront.net/mp4/netflix/{mp4_file}"

!curl {url} -o {mp4_file}

Video(mp4_file, width=640, height=360)

### Upload the sample video to the default Amazon S3 bucket for this notebook

In [ ]:
response = s3h.upload_object(bucket, "contextual_ad", mp4_file) 

## 2. Generate chapter segments based on the dialog in the video

Once the video is uploaded to S3, we will leverage Amazon Transcribe and a foundation model from Bedrock to automatically generate conversational chapter points. This will help us keep track of when conversation topics start and end in the video. The process begins with Amazon Transcribe converting speech to text and generating a transcription. This transcription is then downloaded and formatted into the WebVTT format.

### Use Amazon Transcribe to convert speech to text

This section uses Amazon Transcribe to convert the speech to text and generate a WebVTT output.

If you are getting `AccessDeniedException`, log on to `AWS IAM Console`, find the SageMaker Execution IAM Role, and add the following managed polices:
- AmazonTranscribeFullAccess
- AmazonBedrockFullAccess

Also check out the pricing on [Amazon Transcribe Pricing](https://aws.amazon.com/transcribe/pricing/) in us-east-1 region

In [ ]:
print(f'SageMaker execution IAM Role ARN: {role}')

### Probe the video to get the stream information

In [ ]:
stream_info = ffh.probe_stream(mp4_file)

JSON(stream_info)

### Start the transcription job and wait for it to complete.

In [ ]:
# start transcription job

transcribe_response = trh.transcribe(bucket, "contextual_ad", mp4_file)

### Examine the results from Amazon Transcribe

The response from Amazon Transcribe contains a `results` dictionary with a `transcript` that contains a text-only transcript and a collection of `items` which contain each word and punctuation in the transcript along with a confidence score and timestamp for the item. The response also contains the same transcript formatted as subtitles in either WebVTT or SRT format.  Let's take a look at these outputs.  

We will be using the WebVTT output for our workflow.

**Transcript**

In [ ]:
transcript_filename = trh.download_transcript(transcribe_response, output_dir = video_dir)

JSON(filename=transcript_filename)

**WebVTT Subtitles**

In [ ]:
vtt_filename = trh.download_vtt(transcribe_response, output_dir = video_dir)

!head {vtt_filename}

### Estimate the cost of the transcription job

In [ ]:
duration_ms = stream_info['video_stream']['duration_ms']
transcribe_cost = trh.display_transcription_cost(duration_ms)

### Use an Amazon Bedrock to generate chapters from the subtitles 
Next, the transcript is passed to the Anthropic Claude 3 Haiku model from Amazon Bedrock. The model analyzes the transcript and suggests conversational chapter points in a specific JSON format. In the prompt, we specify that each chapter should contain a start and end timestamp along with a reason describing the topic. The prompts for the Haiku model are shown below:

**System prompt**

```
You are a media operation assistant who analyses movie transcripts in WebVTT 
format and suggest chapter points based on the topic changes in the conversations. 
It is important to read the entire transcripts.
```


**Messages**

```
[
    {
        'content': 'Here is the transcripts in <transcript> tag:\n'
                '<transcript>{transcript}\n</transcript>\n',
        'role': 'user'
    },
    {
        'content': 'OK. I got the transcript. What output format?',
        'role': 'assistant'
    },
    {
        'content': 'JSON format. An example of the output:\n'
                '{"chapters": [{"start": "00:00:10.000", "end": "00:00:32.000", '
                '"reason": "It appears the chapter talks about..."}]}\n',
        'role': 'user'
    },
    {
        'content': '{', 'role': 'assistant'
    }
 ]
```



In [ ]:

conversation_response = brh.analyze_conversations(vtt_filename)

# show the conversation cost
conversation_cost = brh.display_conversation_cost(conversation_response)

### Let's take a look at the conversations that were generated from the transcript 



In [ ]:
conversations = conversation_response['content'][0]['json']

JSON(conversations)

### Generate "chapter points" 

To ensure the model's output accurately reflects the original transcript, the output JSON is post-processed to merge any overlapping chapter timestamps and align the chapter boundaries with the actual caption timestamps from the WebVTT file. 

In [ ]:
## parse the conversation
conversations = conversation_response['content'][0]['json']

## merge overlapped conversation timestamps
chapters = chpt.merge_chapters(conversations['chapters'])

## validate the conversation timestamps against the caption timestamps
captions = chpt.parse_webvtt(vtt_filename)
chapters = chpt.validate_timestamps(chapters, captions)

conversations['chapters'] = chapters

## save the conversations
util.save_to_file(os.path.join(video_dir, 'conversations.json'), conversations)

JSON(conversations)

In [ ]:
estimated_cost = conversation_cost['estimated_cost'] + transcribe_cost['estimated_cost']
estimated_cost = round(estimated_cost, 4)

print('\n')
print('Generating "chapter points"')
print('========================================================================')
print('Transcribe cost:', colored(f"${round(transcribe_cost['estimated_cost'], 4)}", 'green'), f"with duration of {colored(transcribe_cost['duration'], 'green')}s")
print('Bedrock cost:', colored(f"${round(conversation_cost['estimated_cost'], 4)}", 'green'), f"with {colored(conversation_cost['input_tokens'], 'green')} input tokens and {colored(conversation_cost['output_tokens'], 'green')} output tokens.")
print('-----')
print('Estimated cost:', colored(f"${estimated_cost}", 'green'))
print('========================================================================')


## CHECKPOINT

At this point, we have taken the audio part of the video file, run Amazon Transcribe to convert the speech to text, and run Amazon Bedrock (Anthropic Claude 3 Haiku) model to analyze the conversations.

Let's move on to analyzing the visual part of the video.

## 3. Generate a scene grid from video frames

In this section, we will sample the frames from the video and use Amazon Titan Multimodal Embedding (TME) model to help group frames into shots and then group shots into scenes based on visual similarity. Each frame from a scene is used to create a single composite image or "scene grid" that can be used as an input to Amazon Bedrock to understand the scene. 

In this process, we first sample one frame per second, then employ a cosine similarity logic on the adjacent frames to group frame images into shots, which represent camera shot change events. We chose one frame per second for downsampling based on past experiences, but this can be calibrated if you have high-motion, high-frame-rate videos. 

Even after identifying individual camera shots, there may still be too many semantically similar shots depicting the same setting. To further cluster these into distinct scenes, we need to expand our frame comparison beyond just adjacent frames. By looking at similar frames across an expanded time window, we can identify shots that are likely part of the same contiguous scene. We calculate pairwise similarity scores between all frames within a given time window. Frames with similarity scores above a certain threshold are considered part of the same scene group. This process is performed recursively across all frames in a shot. The time window size and similarity threshold are calibrated parameters that can significantly impact scene boundary detection accuracy. In our example, we found a 3-minute time window and 0.85 similarity threshold gave the best scene clustering results across our video samples.

Technically, this scene grouping process is accomplished by first indexing all video frames using TME again and storing the embeddings along with their shot information and timestamps into a vector database, as illustrated in the figure below.  For this notebok, we are using a FAIS vector store to manage embedding locally, but you can use any vector store.  The implementation in Solution Guidance for Media2Cloud on AWS uses Amazon Open Search Serverless for this purpose.

![scene grouping](./static/scene-grouping.png)

**NOTE:** In an automated workflow, this step can be run in parallel to generating chapter points since there is no dependency between the steps.



### Sample frames from the video

In this section, we are extracting 1 frame per second with a resolution of `392x220` from the sample video. Using `392x220` is chosen for a reason and will be discussed in "Generating chapter level contextual information" section.

In [ ]:
jpeg_files = ffh.extract_frames(mp4_file, stream_info, (392, 220))

print(f"Frame extracted: {len(jpeg_files)}")

### Group frames into shots with Amazon Titan Multimodal Embedding

- Generate frame embeddings with Amazon Titan Multimodal Embedding model
- Group frames into shots with cosine similarity

#### Generate frame embeddings with Amazon Titan Multimodal Embedding model

In [ ]:
frame_embeddings = embeddings.batch_generate_embeddings(jpeg_files, output_dir = video_dir)

frame_embeddings_cost = embeddings.display_embedding_cost(frame_embeddings)

#### Group adjacent frames into shots with cosine similarity

In [ ]:
frames_in_shots = frames.group_frames_to_shots(frame_embeddings)

print(f"Number of shots: {len(frames_in_shots)} from {len(frame_embeddings)} frames")

# update shot_id in frame_embeddings dict
for idx, frames_in_shot in enumerate(frames_in_shots):
    for frame_id in frames_in_shot['frame_ids']:
        frame_embeddings[frame_id]['shot_id'] = idx

# save to json file
for file, data in [
    ('frames_in_shots.json', frames_in_shots),
    ('frame_embeddings.json', frame_embeddings)
]:
    output_file = os.path.join(video_dir, file)
    util.save_to_file(output_file, data)

# plot the shot images
frames.plot_shots(video_dir, frame_embeddings, len(frames_in_shots))

print('========')

### Group shots into scenes using similarity search

The previous step (grouping frames to shots) compares the similarity of the adjacent frames. This step compares the frames to the rest of the frame images of the entire content. This allows us to group frame images that are further apart to group the shots into scenes.

We will perform a recursive similarity search against this indexed frame corpus. For each frame, we find all other frames within a 3-minute time window that have greater than 85% contextual similarity based on their vector representations. The shot information for these highly similar frames is recorded. This process iterates across all frames within each shot. Finally, we group the shot information that were mutually identified as highly similar into distinct scene groups. This allows us to segment the initially detected shot boundaries into higher-level semantic scene boundaries based on visual and temporal coherence.

![shots to scenes](./static/shots-to-scenes.png)




In [ ]:
## create an index
dimension = len(frame_embeddings[0]['embedding'])
vector_store = embeddings.create_index(dimension)

## indexing all the frames
embeddings.index_frames(vector_store, frame_embeddings)

## find similar frames for each of the frames and store in the frame_embeddings
for frame in frame_embeddings:
    similar_frames = embeddings.search_similarity(vector_store, frame)
    frame['similar_frames'] = similar_frames

## find all similar frames that are related to the shots and store in the frames_in_shots
for frames_in_shot in frames_in_shots:
    similar_frames_in_shot = frames.collect_similar_frames(frame_embeddings, frames_in_shot['frame_ids'])
    frames_in_shot['similar_frames_in_shot'] = similar_frames_in_shot

    related_shots = frames.collect_related_shots(frame_embeddings, similar_frames_in_shot)
    frames_in_shot['related_shots'] = related_shots

shots_in_scenes = frames.group_shots_in_scenes(frames_in_shots)

# store the scene_id to all structs
for scene in shots_in_scenes:
    scene_id = scene['scene_id']
    shot_min, shot_max = scene['shot_ids']
    # update json files
    for shot_id in range(shot_min, shot_max + 1):
        frames_in_shots[shot_id]['scene_id'] = scene_id
        for frame_id in frames_in_shots[shot_id]['frame_ids']:
            frame_embeddings[frame_id]['scene_id'] = scene_id

# update the json files
# save to json file
for file, data in [
    ('shots_in_scenes.json', shots_in_scenes),
    ('frames_in_shots.json', frames_in_shots),
    ('frame_embeddings.json', frame_embeddings)
]:
    output_file = os.path.join(video_dir, file)
    util.save_to_file(output_file, data)

# plot the scene images
frames.plot_scenes(video_dir, frame_embeddings, len(shots_in_scenes))

print(f"Number of frames: {len(frame_embeddings)}")
print(f"Number of shots: {len(frames_in_shots)}")
print(f"Number of scenes: {len(shots_in_scenes)}")

## 4. Align scene and chapter

At this point, we have separately processed the visual and audio cues from the video. Now, we bring them together and ensure that the transcription chapters align with the scene breaks. The last thing you want is to insert an ad during an ongoing conversation or scene. To create alignment, we will iterate over each conversational chapter, represented by its start and end timestamps, and a text description summarizing the topic. For each chapter, the code identifies the relevant video scenes that overlap or fall within the chapter's timestamp range. The output of this process is a list of chapters, where each chapter contains a list of scene IDs representing the video scenes that align with the corresponding audio conversation. After the alignment process, we have combined visual and audio cues into the final chapters. The breaks we identified are what the system suggested as ideal places for ad insertion. In real-world applications, we recommend surfacing these breaks as suggestions to the operator and having a human-in-the-loop step to confirm the final breaks.


In [ ]:
scenes_in_chapters = frames.group_scenes_in_chapters(
    conversations,
    shots_in_scenes,
    frames_in_shots
)

for scenes_in_chapter in scenes_in_chapters:
    chapter_id = scenes_in_chapter['chapter_id']
    scene_min, scene_max = scenes_in_chapter['scene_ids']

    # update json files
    for scene_id in range(scene_min, scene_max + 1):
        shots_in_scenes[scene_id]['chapter_id'] = chapter_id
        shot_min, shot_max = shots_in_scenes[scene_id]['shot_ids']
        for shot_id in range(shot_min, shot_max + 1):
            frames_in_shots[shot_id]['chapter_id'] = chapter_id
            for frame_id in frames_in_shots[shot_id]['frame_ids']:
                frame_embeddings[frame_id]['chapter_id'] = chapter_id

# update the json files
for file, data in [
    ('scenes_in_chapters.json', scenes_in_chapters),
    ('shots_in_scenes.json', shots_in_scenes),
    ('frames_in_shots.json', frames_in_shots),
    ('frame_embeddings.json', frame_embeddings),
]:
    output_file = os.path.join(video_dir, file)
    util.save_to_file(output_file, data)

# plot the chapter images
frames.plot_chapters(video_dir, frame_embeddings, len(scenes_in_chapters))

print('-----')

## 5. Generate chapter level contextual information 

The last step is to send both the visually and audio-aligned data to Claude 3 Haiku to generate contextual information for each chapter. This is an innovative approach that takes advantage of the multimodal capabilities of the Claude 3 family of models. From our testing, these models have demonstrated the ability to capture minute details from large images and follow image sequences when provided with appropriate instructions.

To prepare the input for Claude3 Haiku, we first assemble video frames associated with each chapter and create a composite image grid. Through our experimentation, we have found that the optimum image grid ratio is 7 rows by 4 columns, which will assemble a **1568 x 1540 pixel image** that fits under Claude's 5 MB image file size limit while still preserving enough detail in each individual frame tile. Furthermore, you can also assemble multiple images if needed.

<img src="static/scene-grid.png" width="50%" >

Subsequently, the composite images and transcription are fed into the prompt to generate descriptions, sentiment, and other relevant information in a single query to the Claude3 Haiku model. Not only that, but we can adapt this approach to any taxonomy or custom labeling use cases without the need to train a model each time. This is where the true power of this approach lies. The final output can be presented to a human reviewer for final confirmation if needed. Here is an example of a composite image grid and the corresponding contextual output for a specific chapter.

In [ ]:
def get_chapter_frames(frame_embeddings, scenes_in_chapters):
    num_chapters = len(scenes_in_chapters)
    chapters_frames = [{
        'chapter_id': i,
        'text': '',
        'frames': [],
    } for i in range(num_chapters)]

    for frame in frame_embeddings:
        chapter_id = frame['chapter_id']
        file = frame['file']
        chapters_frames[chapter_id]['frames'].append(file)
        chapters_frames[chapter_id]['text'] = scenes_in_chapters[chapter_id]['text']
        
    return chapters_frames

### Create composte images and use Anthropic Claude to generate contextual information for each chapter

**System Prompt:**
```
You are a media operation engineer. Your job is to review a portion of a video content presented in a sequence of consecutive images. Each image also contains a sequence of frames presented in a 4x7 grid reading from left to right and then from top to bottom. You may also optionally be given the conversation of the scene that helps you to understand the context. You are asked to provide the following information: a detail description to describe the scene, identify the sentiment, brands and logos that may appear in the scene, and five most relevant tags from the scene. It is important to return the results in JSON format and also includes a confidence score from 0 to 100. Skip any explanation.
```

**Prompt Body:**

```json
{'anthropic_version': 'bedrock-2023-05-31',
 'max_tokens': 4096,
 'messages': [{'content': [{'text': 'Here are X images containing frame '
                                    'sequence that describes a scene.',
                            'type': 'text'},
                           {'source': {'data': <IMAGE_GRID>,
                                       'media_type': 'image/jpeg',
                                       'type': 'base64'},
                            'type': 'image'}],
               'role': 'user'},
              {'content': 'Got the images. Do you have the conversation of the '
                          'scene?',
               'role': 'assistant'},
              {'content': 'No conversation.', 'role': 'user'},
              {'content': 'OK. Do you have other information to provdie?',
               'role': 'assistant'},
              {'content': [{'text': 'Here is a list of Sentiments in '
                                    '<sentiment> tag:\n'
                                    '<sentiment>\n'
                                    <SENTIMENT_LABELS>
                                    '</sentiment>\n'
                                    'Only answer the sentiment from this list.',
                            'type': 'text'}],
               'role': 'user'},
              {'content': 'OK. What output format?', 'role': 'assistant'},
              {'content': 'Return JSON format. An example of the output:\n'
                          <JSON_EXAMPLE>
               'role': 'user'},
              {'content': '{', 'role': 'assistant'}]
```

**Output Example:**
```json
{
    "description": {
        "text": "The scene describes...",
        "score": 98
    }, 
    "sentiment": {
        "text": "Positive", 
        "score": 90
    }, 
    "brands_and_logos": [
        {"text": "Amazon", "score": 95}, 
        {"text": "Nike", "score": 85}
    ], 
    "relevant_tags": [
        {"text": "BMW", "score": 95}
    ]
}
```

In [ ]:
total_usage = {
    'input_tokens': 0,
    'output_tokens': 0,
}

frames_in_chapters = get_chapter_frames(frame_embeddings, scenes_in_chapters)

for frames_in_chapter in frames_in_chapters:
    chapter_id = frames_in_chapter['chapter_id']
    text = frames_in_chapter['text']
    ch_frames = frames_in_chapter['frames']

    composite_images = frames.create_composite_images(ch_frames)
    num_images = len(composite_images)

    for j in range(num_images):
        composite_image = composite_images[j]
        print(f"Chapter #{chapter_id:02d}: {j + 1} of {num_images} composite images")
        w, h = composite_image.size
        scaled = composite_image.resize((w // 4, h // 4))
        display(scaled)

    contextual_response = brh.get_contextual_information(composite_images, text)
    
    # close the images
    for composite_image in composite_images:
        composite_image.close()

    usage = contextual_response['usage']
    contextual = contextual_response['content'][0]['json']

    # save the contextual to the chapter
    scenes_in_chapters[chapter_id]['contextual'] = {
        'usage': usage,
        **contextual
    }

    total_usage['input_tokens'] += usage['input_tokens']
    total_usage['output_tokens'] += usage['output_tokens']

    print(f"==== Chapter #{chapter_id:02d}: Contextual information ======")
    for key in ['description', 'sentiment']:
        print(f"{key.capitalize()}: {colored(contextual[key]['text'], 'green')} ({contextual[key]['score']}%)")

    for key in ['brands_and_logos', 'relevant_tags']:
        items = ', '.join([item['text'] for item in contextual[key]])
        if len(items) == 0:
            items = 'None'
        print(f"{key.capitalize()}: {colored(items, 'green')}")
    print(f"================================================\n\n")

output_file = os.path.join(video_dir, 'scenes_in_chapters.json')
util.save_to_file(output_file, scenes_in_chapters)

contextual_cost = brh.display_contextual_cost(total_usage)

### Total estimated cost

In [ ]:
total_estimated_cost = 0

for estimated_cost in [transcribe_cost, conversation_cost, frame_embeddings_cost, contextual_cost]:
    total_estimated_cost += estimated_cost['estimated_cost']
total_estimated_cost = round(total_estimated_cost, 4)

print('\n')
print('\n== Generating chapter points ===========================================\n')
print('Transcribe cost:', colored(f"${round(transcribe_cost['estimated_cost'], 4)}", 'green'), f"with duration of {colored(transcribe_cost['duration'], 'green')}s")
print('Claude cost:', colored(f"${round(conversation_cost['estimated_cost'], 4)}", 'green'), f"with {colored(conversation_cost['input_tokens'], 'green')} input tokens and {colored(conversation_cost['output_tokens'], 'green')} output tokens.")
print('\n== Generating image embeddings =========================================\n')
print('Titan cost:', colored(f"${round(frame_embeddings_cost['estimated_cost'], 4)}", 'green'), f"with {colored(frame_embeddings_cost['num_embeddings'], 'green')} embeddings.")
print('\n== Chapter contextual information ======================================\n')
print('Claude cost:', colored(f"${round(contextual_cost['estimated_cost'], 4)}", 'green'), f"with {colored(contextual_cost['input_tokens'], 'green')} input tokens and {colored(contextual_cost['output_tokens'], 'green')} output tokens.")
print('\n========================================================================\n')
print('Total estimated cost:', colored(f"${total_estimated_cost}", 'green'))
print('\n========================================================================')
